In [1]:
import os
import csv

import numpy as np
import scipy.stats as stats
import numba
from tqdm import tqdm

from voi import compute_EVPI, compute_EVII

from utils import cached

In [2]:
np.random.seed(0) # seed sampling

# 1. Define action space
maintenance_freqs = np.arange(13) # number of maintenance operations per year

# 2. Define sampling function for uncertain parameter(s)
@numba.njit(fastmath=True)
def discritise_samples(samples, discr_points):
    return np.array([discr_points[np.argmin(np.abs(discr_points-s))] for s in samples])

def fast_discritise_samples(samples, discr_points,bins):
    return discr_points[np.digitize(samples,bins)]

def prior_theta_sampler(n_samples):
    """Sample vectors of uncertain parameter values from prior distributions."""

    # set up parameters
    # =================
    # sample discretisation used to allow caching for compuational efficiency
    discr_delta = 1e-2

    # alpha - truncated Gaussian [0,1)
    alpha_mu = 1e-2
    alpha_sigma = 0.25
    alpha_discr_points = np.arange(0,1-discr_delta,discr_delta)
    # epsilon - log Normal
    epsilon_mu = 0
    epsilon_sigma = 0.1
    epsilon_discr_points = np.arange(-1,1,discr_delta)
    # SPF' - Gaussian
    spf_dash_mu = 2.9
    spf_dash_sigma = 0.167
    spf_dash_discr_points = np.arange(2,4,discr_delta)
    # elec unit cost (£/kWh) - Gaussian
    elec_unit_cost_mu = 0.326
    elec_unit_cost_sigma = 0.016
    # annual load - Gaussian
    annual_load_mu = 12560000
    annual_load_sigma = 1358000

    theta_matrix = np.vstack([
        discritise_samples(stats.truncnorm(-1*alpha_mu/alpha_sigma,(1-alpha_mu)/alpha_sigma,loc=alpha_mu,scale=alpha_sigma).rvs(n_samples),alpha_discr_points), # alpha
        discritise_samples(stats.norm(loc=epsilon_mu,scale=epsilon_sigma).rvs(n_samples),epsilon_discr_points), # epsilon
        discritise_samples(stats.norm(loc=spf_dash_mu,scale=spf_dash_sigma).rvs(n_samples),spf_dash_discr_points), # spf_dash
        stats.norm(loc=elec_unit_cost_mu,scale=elec_unit_cost_sigma).rvs(n_samples), # elec_unit_cost
        stats.norm(loc=annual_load_mu,scale=annual_load_sigma).rvs(n_samples), # annual_load
    ])
    thetas = [theta_matrix[:,i] for i in range(theta_matrix.shape[1])] # unpack thetas

    return thetas

def simplified_prior_theta_sampler(n_samples):

     # set up parameters
    # =================
    # sample discretisation used to allow caching for compuational efficiency
    discr_delta = 1e-2

    # alpha - truncated Gaussian [0,1)
    alpha_mu = 1e-2
    alpha_sigma = 0.25
    alpha_discr_points = np.arange(0,1-discr_delta,discr_delta)
    alpha_bins = np.array([-1*np.inf, *[np.mean(alpha_discr_points[i:i+2]) for i in range(len(alpha_discr_points)-1)], np.inf])
    # epsilon - log Normal
    epsilon_mu = 0
    epsilon_sigma = 0.1
    epsilon_discr_points = np.arange(-1,1,discr_delta)
    epsilon_bins = np.array([-1*np.inf, *[np.mean(epsilon_discr_points[i:i+2]) for i in range(len(epsilon_discr_points)-1)], np.inf])
    # SPF' - Gaussian
    spf_dash_mu = 2.9
    spf_dash_sigma = 0.167
    spf_dash_discr_points = np.arange(2,4,discr_delta)
    spf_dash_bins = np.array([-1*np.inf, *[np.mean(spf_dash_discr_points[i:i+2]) for i in range(len(spf_dash_discr_points)-1)], np.inf])

    theta_matrix = np.vstack([
        fast_discritise_samples(stats.truncnorm(-1*alpha_mu/alpha_sigma,(1-alpha_mu)/alpha_sigma,loc=alpha_mu,scale=alpha_sigma).rvs(n_samples),alpha_discr_points,alpha_bins), # alpha
        fast_discritise_samples(stats.norm(loc=epsilon_mu,scale=epsilon_sigma).rvs(n_samples),epsilon_discr_points,epsilon_bins), # epsilon
        fast_discritise_samples(stats.norm(loc=spf_dash_mu,scale=spf_dash_sigma).rvs(n_samples),spf_dash_discr_points,spf_dash_bins)
    ])

    return theta_matrix.T


def prior_theta_and_partial_perfect_z_sampler(n_samples, perfect_info_params=None):
    """Sample thetas and zs for partial perfect information for specified parameters."""

    thetas = [t[:3] for t in prior_theta_sampler(n_samples)]

    parameters = ['alpha', 'epsilon', 'spf_dash']
    if perfect_info_params is None:
        perfect_info_params = {param: False for param in parameters}
    measure_params = np.array(list(perfect_info_params.values()),dtype=bool)

    zs = [np.where(measure_params==True, t, np.nan) for t in thetas]

    return thetas, zs

def partial_perfect_info_theta_sampler(measurement, n_samples):
    """Sample thetas for case of partial perfect information with specified measurement."""

    thetas = [t[:3] for t in prior_theta_sampler(n_samples)]
    thetas_with_perfect_info = [np.where(np.isnan(measurement), t, measurement) for t in thetas]

    return thetas_with_perfect_info


# 3. Define system dynamics (intermediate computations)
@numba.njit(fastmath=True)
def compute_beta(maint_freq, epsilon):
    beta_a = 0.05
    beta_b = 2.5
    gamma = 1.4
    return ((beta_a*maint_freq**gamma)/(beta_b+maint_freq**gamma))*(1+epsilon)

@numba.njit(fastmath=True)
def compute_spf(alpha, beta, spf_dash):
    """Compute seasonal performance factor."""
    return spf_dash*(1-alpha)*(1+beta)

# 4. Define utility function
@numba.njit(fastmath=True)
def utility(maint_freq, theta):

    # unpack theta
    alpha = theta[0]
    epsilon = theta[1]
    spf_dash = theta[2]
    elec_unit_cost = theta[3] # £/kWh
    annual_load = theta[4] # kWh/year

    # set up cost parameters
    maint_unit_cost = 552.5*30 # £ per maintainence operation on 10 ASHPs (originally 4 for 1.75GWh/year)

    # compute spf
    beta = compute_beta(maint_freq, epsilon)
    spf = compute_spf(alpha, beta, spf_dash)

    # compute cost contributions
    maintenance_cost = maint_unit_cost*maint_freq # £/year
    electricity_cost = annual_load*elec_unit_cost/spf # £/year

    return -1*(maintenance_cost+electricity_cost) # utility [+£/year]

@numba.njit(fastmath=True)
def simplified_utility(maint_freq, theta):
    """Utility function for simplified problem with deterministic
    electricity price and heating load.
    Use theta = [alpha, elec_unit, annual_load]."""

    elec_unit_cost = 0.326
    annual_load = 12560000

    # unpack theta
    alpha = theta[0]
    epsilon = theta[1]
    spf_dash = theta[2]

    # set up cost parameters
    maint_unit_cost = 552.5*30 # £ per maintainence operation on 10 ASHPs (originally 4 for 1.75GWh/year)

    # compute spf
    beta = compute_beta(maint_freq, epsilon)
    spf = compute_spf(alpha, beta, spf_dash)

    # compute cost contributions
    maintenance_cost = maint_unit_cost*maint_freq # £/year
    electricity_cost = annual_load*elec_unit_cost/spf # £/year

    return -1*(maintenance_cost+electricity_cost) # utility [+£/year]

In [3]:
thetas = simplified_prior_theta_sampler(int(1e6))

IndexError: index 99 is out of bounds for axis 0 with size 99

In [ ]:
print(thetas)

[[ 0.19 -0.05  2.8 ]
 [ 0.27  0.05  2.99]
 [ 0.22 -0.04  2.9 ]
 ...
 [ 0.12 -0.07  3.27]
 [ 0.18 -0.07  3.18]
 [ 0.43  0.02  3.02]]


In [ ]:
maint_freq = 2

In [ ]:
np.array([simplified_utility(maint_freq, t) for t in tqdm(thetas)])

  0%|          | 0/1000000 [00:00<?, ?it/s]

100%|██████████| 1000000/1000000 [00:01<00:00, 687621.71it/s]


array([-1795522.93599828, -1859818.01628238, -1799754.86784107, ...,
       -1422872.87374398, -1566769.62781319, -2351066.28328059])